In [1]:
from testing.test_suite import run_all_tests

run_all_tests()

test_blank_regex (testing.test_suite.RegexTests.test_blank_regex) ... ok
test_regex_edge_cases (testing.test_suite.RegexTests.test_regex_edge_cases) ... ok
test_regex_no_match (testing.test_suite.RegexTests.test_regex_no_match) ... ok
test_regex_with_spaces (testing.test_suite.RegexTests.test_regex_with_spaces) ... ok
test_time_coversion_blank (testing.test_suite.TimeConversionTests.test_time_coversion_blank) ... ok
test_time_coversion_standard (testing.test_suite.TimeConversionTests.test_time_coversion_standard) ... ok
test_time_rare_formatting (testing.test_suite.TimeConversionTests.test_time_rare_formatting) ... ok
test_extract_text_with_paddle_blank (testing.test_suite.TextExtractionTests.test_extract_text_with_paddle_blank) ... ok
test_extract_text_with_paddle_none (testing.test_suite.TextExtractionTests.test_extract_text_with_paddle_none) ... ok
test_extract_text_with_paddle_valid (testing.test_suite.TextExtractionTests.test_extract_text_with_paddle_valid) ... ok
test_extract_tim

In [ ]:
from moviepy.editor import VideoFileClip

def extract_clip(video_path, start_time, output_path):
    """
    Extracts a 10-second clip from the specified video starting at `start_time`.
    
    :param video_path: Path to the source video file.
    :param start_time: Start time for the clip in seconds.
    :param output_path: Path where the extracted clip will be saved.
    """
    try:
        video = VideoFileClip(video_path)
        end_time = start_time + 10
        clip = video.subclip(start_time, end_time)
        clip.write_videofile(output_path, codec="libx264")
        print(f"Clip successfully extracted to {output_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

in_path = r"demo\17590_01-23-2016_2_Golden State Warriors_9_Indiana Pacers_period4.mp4"
out_path = r"demo/demo_17590_period_4.mp4"
extract_clip(in_path, 60, out_path)  # Extracts a clip starting at 1 minute


In [ ]:
from temporal_grounding import extract_timestamps_from_video

vp = r"demo\demo_17590_period_4.mp4"
op = "demo/timestamps.json"
extract_timestamps_from_video(vp, op)

# TODO
# 1. Post Process Timestamps
# 2. Map time-remaining to stavulog
# 3. visualize results

In [ ]:
testing/assets/blank_images/black.png

In [ ]:
from ultralytics import YOLO
from paddleocr import PaddleOCR

MODEL_PATH = r"models/yolo/weights/tr_roi_finetune_130_large.pt"


class Models:

    yolo = yolo = YOLO(MODEL_PATH)
    paddle_ocr = PaddleOCR(use_angle_cls=True,
                           lang='en',
                           show_log=False,
                           det_db_score_mode='slow',
                           ocr_version='PP-OCRv4',
                           rec_algorithm='SVTR_LCNet',
                           drop_score=0.8,
                           )


In [ ]:
import cv2
import torch
import json
import os
import re
import time
import numpy as np

from tqdm import tqdm
from PIL import Image
from typing import List

from viz import visualize_timestamps
from utilities.models import Models
from utilities.constants import *

def extract_text_with_paddle(image) -> List[str]:
    """
    Returns a [str] containing all words found in a
    provided PIL image.
    """

    def resize_with_opencv(image_pil, new_size):
        image_np = np.array(image_pil)
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
        resized_image = cv2.resize(image_np, new_size)
        return resized_image

    if image is None:
        return []
    image = Image.fromarray(image)
    image = image.convert("RGB")
    ideal_height = 100
    scale_factor = ideal_height / image.height
    new_size = (int(image.width * scale_factor),
                int(image.height * scale_factor))
    image = resize_with_opencv(image, new_size)
    results = []
    raw_result = Models.paddle_ocr.ocr(image, det=False, rec=False, cls=False)
    text_arr = raw_result[1]
    for pred in text_arr:
        word = pred[0]
        results.append(word)
    return results

In [ ]:
print("read")
image = cv2.imread(r"/Users/leviharris/Documents/contextualized-shot-quality-estimation/testing/assets/example_cropped_rois/time_remaining/time_remaining_1.png")
results = extract_text_with_paddle(image)
assert results == ["2:41"]

In [ ]:
import os
from temporal_grounding import process_dir

vid_dir = r"demo/video_dir"
logs_dir = "demo/timestamps_dir"
viz_dir = "demo/viz_dir"

process_dir(vid_dir, logs_dir, viz_dir)

In [ ]:
from viz import visualize_timestamps

visualize_timestamps(
    video_path='demo/video_dir/707_01-17-2016_2976_Memphis Grizzlies_3173_New York Knicks_period1.mp4',
    viz_path='demo/viz_dir/viz.mp4',
    timestamps_path='demo/timestamps_dir/707_01-17-2016_2976_Memphis Grizzlies_3173_New York Knicks_period1.json'
)

In [ ]:
import os
import time
from temporal_grounding import *
from viz import *

video_path = r'demo/video_dir/707_01-17-2016_2976_Memphis Grizzlies_3173_New York Knicks_period1.mp4'
# demo_dir = r'demo'
# start = time.time()
# extract_timestamps_from_video(
#     video_path, 
#     os.path.join(demo_dir, 'example.json')
#     )
# print("--- %s seconds ---" % (time.time() - start))

In [ ]:
import os
import time
from temporal_grounding import *
from viz import *

video_path = r'demo/video_dir/707_01-17-2016_2976_Memphis Grizzlies_3173_New York Knicks_period1.mp4'
timestamps_path = r"demo/example.json"
viz_path = r"demo/viz_dir/viz.avi"
visualize_timestamps(video_path, timestamps_path, viz_path)

In [ ]:
import math

def interpolate_timestamps(timestamps_path: str, output_path) -> None:
    """
    Interpolates time remaining values in a JSON file containing timestamp data.

    Args:
    file_path (str): The file path of the input JSON file with timestamp data.
    output_path (str): The file path for the output JSON file with interpolated timestamps.

    The function reads the timestamp data, interpolates the 'time_remaining' values, and writes
    the updated data to a new JSON file.
    """

    # Load data from the file
    try:
        with open(timestamps_path, 'r') as file:
            data = json.load(file)
    except Exception as e:
        print(f"Error reading file {timestamps_path}: {e}")
        return
    
    original_time_values = []

    # Extract time_remaining values and handle None values
    for key in data:
        time_remaining = data[key].get('time_remaining', -1)
        original_time_values.append(time_remaining if time_remaining is not None else -1)

    last_seen_valid_time = -1
    consecutive_frame_count = 0
    interpolated_values = []

    for value in original_time_values:
        if value > 0:
            if consecutive_frame_count == 0 or consecutive_frame_count > 30:
                last_seen_valid_time = value
                consecutive_frame_count = 0
            else:
                multiplier = math.floor((consecutive_frame_count / 30) * 25)
                interpolated_value = round(value - (multiplier / 25), 2)
                interpolated_values.append(interpolated_value)
            consecutive_frame_count += 1
        else:
            interpolated_values.append(None)

    # Update the data dictionary with interpolated values
    for key, interpolated_value in zip(data, interpolated_values):
        data[key]['time_remaining'] = interpolated_value

    try:
        with open(output_path, "w") as outfile:
            json.dump(data, outfile, indent=4)
    except Exception as e:
        print(f"Error writing file {output_path}: {e}")

ts_path = r"demo/example.json"
out_path = "demo/example_post_processed.json"
interpolate_timestamps(ts_path, out_path)